In [39]:
import numpy as np
import pandas as pd
from random import randint,sample
import time

import warnings
warnings.filterwarnings('ignore')

In [40]:
## cloning the repositry from the github to make the file easily shareable
!git clone https://github.com/guru-narayana/NNFL-assignment

fatal: destination path 'NNFL-assignment' already exists and is not an empty directory.


In [41]:
## loading the data
data_frame = pd.read_excel("/content/NNFL-assignment/data5.xlsx").sample(frac = 1)
data = data_frame.to_numpy()

## seperating input and output data
input_prenorm_data = data[:,0:data.shape[1]-1]
op = data[:,data.shape[1]-1]

## normalising the data
max_data    = np.amax(input_prenorm_data,axis=0) ## max  values in each column
input_data  = input_prenorm_data/max_data
output_data = np.zeros((len(op),3))

#Auto encoding
for i in range(len(op)):
  output_data[i,int(op[i]-1)] = 1

In [42]:
trim_point_1 = int(input_data.shape[0]*0.7)
trim_point_2 = int(input_data.shape[0]*0.9)


input_train_data      = input_data[0:trim_point_1,:]
input_validation_data = input_data[trim_point_1:trim_point_2,:]
input_test_data       = input_data[trim_point_2:input_data.shape[0],:]

output_train_data      = output_data[0:trim_point_1]
output_validation_data = output_data[trim_point_1:trim_point_2]
output_test_data       = output_data[trim_point_2:input_data.shape[0]]

In [43]:
def perceptron(W,X,B):
  P = (X@W.T) + B
  return 1/(1 + np.exp(-P))
def auto_encode(input_data,alpha,lamda,iterations):
  W1 = np.random.rand(input_data.shape[1]-1,input_data.shape[1])
  W2 = np.random.rand(input_data.shape[1],input_data.shape[1]-1)
  B1 = np.random.rand(input_data.shape[1]-1)
  B2 = np.random.rand(input_data.shape[1])
  for i in range(iterations):
    H1 = perceptron(W1,input_data,B1)
    H2 = perceptron(W2,H1,B2)
    error  = (H2-input_data)
    delta2 = error*H2*(1-H2)
    delta1 = np.dot(delta2,W2)*H1*(1-H1)
    W2 -= alpha*(np.dot(delta2.T,H1) + lamda*W2)
    W1 -= alpha*(np.dot(delta1.T,input_data) + lamda*W1)
    B2 -= alpha*np.sum(delta2,axis=0)
    B1 -= alpha*np.sum(delta1,axis=0)
    #print(np.sum(error**2)/(error.shape[1] + error.shape[0]))
  return W1,B1,perceptron(W1,input_data,B1)

In [44]:
def confiusion_matrix_func(outpt,Y):
  pred = np.argmax(outpt, axis=1)+1
  output_test_data = np.argmax(Y, axis=1)+1
  confusion_matrix = np.zeros((3,3))
  count = 0
  for x in range(len(pred)):
    if(pred[x] == output_test_data[x]):
      count+=1
    if(pred[x] == 1):
      if(output_test_data[x] == 1):
        confusion_matrix[0][0]+=1
      elif(output_test_data[x] == 2):
        confusion_matrix[1][0]+=1
      elif(output_test_data[x] == 3):
        confusion_matrix[2][0]+=1
    elif(pred[x] == 2):
      if(output_test_data[x]   == 1):
        confusion_matrix[0][1]+=1
      elif(output_test_data[x] == 2):
        confusion_matrix[1][1]+=1
      elif(output_test_data[x] == 3):
        confusion_matrix[2][1]+=1
    elif(pred[x] == 3):
      if(output_test_data[x]   == 1):
        confusion_matrix[0][2]+=1
      elif(output_test_data[x] == 2):
        confusion_matrix[1][2]+=1
      elif(output_test_data[x] == 3):
        confusion_matrix[2][2]+=1
  c1 = confusion_matrix[0][0]/np.sum(confusion_matrix[0][:])
  c2 = confusion_matrix[1][1]/np.sum(confusion_matrix[1][:])
  c3 = confusion_matrix[2][2]/np.sum(confusion_matrix[2][:])
  A  = count/len(pred)
  return c1,c2,c3,A

In [53]:
def stacked_auto_encoder(alpha,lamda,epochs):
  
  ### pretraining 
  W1,B1,H1 = auto_encode(input_train_data,6e-2,1e-5,10000)
  W2,B2,H2 = auto_encode(H1,6e-2,1e-5,10000)
  W3,B3,H3 = auto_encode(H2,3e-2,1e-5,10000)


  ###fine tuning 
  W4 = np.random.rand(H3.shape[1]-1,H3.shape[1])
  B4 = np.random.rand(H3.shape[1]-1)
  alpha1 = 100*alpha
  for i in range(epochs):
    H4 = perceptron(W4,H3,B4)
    error  = (H4-output_train_data)
    delta4 = error*H4*(1-H4)
    delta3 = np.dot(delta4,W4)*H3*(1-H3)
    delta2 = np.dot(delta3,W3)*H2*(1-H2)
    delta1 = np.dot(delta2,W2)*H1*(1-H1)
    W4 -= alpha1*(np.dot(delta4.T,H3) + lamda*W4)
    W3 -= alpha*(np.dot(delta3.T,H2) + lamda*W3)
    W2 -= alpha*(np.dot(delta2.T,H1) + lamda*W2)
    W1 -= alpha*(np.dot(delta1.T,input_train_data) + lamda*W1)
    B4 -= alpha1*np.sum(delta4,axis=0)
    B3 -= alpha*np.sum(delta3,axis=0)
    B2 -= alpha*np.sum(delta2,axis=0)
    B1 -= alpha*np.sum(delta1,axis=0)
  H_test = perceptron(W4,perceptron(W3,perceptron(W2,perceptron(W1,input_test_data,B1),B2),B3),B4)
  c1,c2,c3,A = confiusion_matrix_func(H_test,output_test_data)
  print("class_1  Accuracy  : "+ str( c1))
  print("class_2  Accuracy  : "+ str( c2))
  print("class_3  Accuracy  : "+ str( c3))
  print( "Overall Accuracy  : "+ str( A))
  print("\n")


In [52]:
stacked_auto_encoder(2e-5,1,1000)

class_1  Accuracy  : 0.7142857142857143
class_2  Accuracy  : 1.0
class_3  Accuracy  : 1.0
Overall Accuracy  : 0.9047619047619048


